In [16]:
from lasagne.layers import InputLayer, Conv2DLayer, MaxPool2DLayer, DenseLayer, GlobalPoolLayer, Upscale2DLayer
from lasagne.layers import ElemwiseSumLayer, NonlinearityLayer, SliceLayer, ConcatLayer, ScaleLayer
from lasagne.layers import dropout, batch_norm
from lasagne.nonlinearities import rectify, softmax, sigmoid
from lasagne.init import GlorotNormal, GlorotUniform, HeUniform, HeNormal
from lasagne.objectives import squared_error, categorical_crossentropy, categorical_accuracy, binary_accuracy
import lasagne
import theano.tensor as T
import numpy as np
import random
import theano
import os
import pandas as pd
import cv2
import re
import sys
import json
import benchmark as bm
import utee
from fusion.fcn1.adapter import adapter as adapter1
from fusion.fcn2.adapter import adapter as adapter2
from fusion.fcn3.adapter import adapter as adapter3
from fusion.fcn4.adapter import adapter as adapter4
from fusion.fcn5.adapter import adapter as adapter5
from fusion.fcn6.adapter import adapter as adapter6

# Configuration

In [8]:
lr = 0.001
n_epoches = 20
loss_epoches = []
snapshot_root = 'fusion_snapshot'
n_train_samples = len(x_data_train)
n_val_samples = len(x_data_val)

# Build model

In [9]:
# all adapters
adapters = []
adapters.append(adapter1((48, 48), 'fusion/fcn1/96.npz'))
# adapters.append(adapter2((48, 48), 'fusion/fcn2/92.npz'))
# adapters.append(adapter3((48, 48), 'fusion/fcn3/52.npz'))
# adapters.append(adapter4((48, 48), 'fusion/fcn4/470.npz'))
# adapters.append(adapter5((48, 48), 'fusion/fcn5/280.npz'))
# adapters.append(adapter6((48, 48), 'fusion/fcn6/114.npz'))

# input tensor
pred = T.tensor4('pred')
location = T.vector('location')
resolution = T.matrix('resolution')
target_volume = T.fscalar('volume')


# fusion layers
l_in = InputLayer(shape=(None, len(adapters), fixed_size[0], fixed_size[1]), input_var = pred)
mid1 = batch_norm(Conv2DLayer(l_in, num_filters=32, filter_size=(3, 3), nonlinearity=rectify, W=HeNormal()))
mid2 = Conv2DLayer(mid1, num_filters=1, filter_size=(1, 1), W=HeNormal())
l_out = GlobalPoolLayer(mid2)

# area, 1d vector
train_area = lasagne.layers.get_output(l_out).flatten()
val_area = lasagne.layers.get_output(l_out, deterministic=True).flatten()

# predict volume, 0d scalar
train_pred_volume = utee.build_volume2(train_area, location, resolution, fixed_size) 
val_pred_volume = utee.build_volume2(val_area, location, resolution, fixed_size)

# loss, 0d scalar
train_loss = T.abs_(train_pred_volume - target_volume).mean() / 600.
val_loss = T.abs_(val_pred_volume - target_volume).mean() / 600.

# params
params = lasagne.layers.get_all_params(l_out, trainable=True)
updates = lasagne.updates.nesterov_momentum(train_loss, params, learning_rate=lr, momentum=0.9)

train_fn = theano.function(
    [pred, location, resolution, target_volume],
    train_loss,
    updates = updates
)
val_fn = theano.function(
    [pred, location, resolution, target_volume],
    val_loss
)
test_fn = theano.function(
    [pred, location, resolution],
    [val_area, val_pred_volume]
)

area_fn = theano.function(
    [pred],
    val_area
)

resuming snapshot from fusion/fcn1/96.npz


# Training and validating

In [13]:
print("Training and validating precedure begin")
for cur_epoch in range(n_epoches+1):
    print("epoch {}/{} begin".format(cur_epoch, n_epoches))
    
    if cur_epoch > 0:
        print(".training, {} samples to go".format(n_train_samples))
        losses_data_train = []
        for j in range(n_train_samples):
            x_e = x_data_train[j].astype('float32')
            preds = []
            for adapter in adapters:
                preds.append(adapter.convert(x_e))
            pred_e = np.concatenate(preds, axis=1)
            location_e = location_data_train[j].astype('float32')
            resolution_e = resolution_data_train[j].astype('float32')
            volume_e = volume_data_train[j].astype('float32')
            loss_data_train = train_fn(pred_e, location_e, resolution_e, volume_e)
            losses_data_train.append(loss_data_train)
        print(".training loss: {}".format(np.mean(loss_data_train)))
    
    print(".validating, {} samples to go".format(n_val_samples))
    losses_data_val = []
    for i in range(n_val_samples):
        volume_min_e = volume_data_val[2*i].astype('float32')
        volume_max_e = volume_data_val[2*i+1].astype('float32')
        pred_volumes_data = []
        for j in range(len(x_data_val[i])):
            x_e = x_data_val[i][j].astype('float32')
            preds = []
            for adapter in adapters:
                preds.append(adapter.convert(x_e))
            pred_e = np.concatenate(preds, axis=1)
            location_e = location_data_val[i][j].astype('float32')
            resolution_e = resolution_data_val[i][j].astype('float32')
            _, pred_volume_data = test_fn(pred_e, location_e, resolution_e)
            pred_volumes_data.append(pred_volume_data)
        volume_min_pred = np.min(pred_volumes_data)
        volume_max_pred = np.max(pred_volumes_data)
        loss_min_data = np.abs(volume_min_pred - volume_min_e) / 600
        loss_max_data = np.abs(volume_max_pred - volume_max_e) / 600
        loss_data_val = (loss_min_data + loss_max_data) / 2.0
    print(".validating loss: {}".format(np.mean(losses_data_val)))
    
    loss_epoches.append(np.mean(losses_data_val))
    
    if np.isnan(np.mean(losses_data_train)) or np.isnan(np.mean(loss_data_val)):
        print(".detect nan, break and stop")
        break
        
    cur_snapshot_path = os.path.join(snapshot_root, str(cur_epoch) + '.npz')
    print("snapshot to {}".format(cur_snapshot_path))
    np.savez(cur_snapshot_path, *lasagne.layers.get_all_param_values(l_out))
print("Training done!")

Training and validating precedure begin
epoch 0/2 begin
.validating, 200 samples to go
.validating loss: 0.0390633158386
snapshot to fusion_snapshot/0.npz
epoch 1/2 begin
.training, 800 samples to go
.training loss: 0.0224609132856
.validating, 200 samples to go
.validating loss: 0.0384977124631
snapshot to fusion_snapshot/1.npz
epoch 2/2 begin
.training, 800 samples to go
.training loss: 0.0219819135964
.validating, 200 samples to go
.validating loss: 0.038855869323
snapshot to fusion_snapshot/2.npz
Training done!


# Saving parameters

In [21]:
idx = np.argmin(loss_epoches)
with open('./SETTINGS.json', 'r') as f:    
    data = json.load(f)
print("add info {} to SETTINGS.json".format(snapshot_root, str(idx) +'.npz'))
data['FUSION_SNAPSHOT'] = os.path.join()
with open('./SETTINGS2.json', 'w') as f:
    json.dump(data, f)

{u'OUT_VALIDATE_DATA_PATH': u'sdf', u'OUT_TRAIN_DATA_PATH': u'df', u'IN_VALIDATE_DATA_PATH': u'sdf', 'FUSION_SNAPSHOT': 'fusion_snapshot/1.npz', u'IN_TRAIN_DATA_PATH': u'sdf'}


# Testing

In [5]:
# print("-----------------predicting---------------------")
# losses_data = []
# n_samples = int(len(x_data) * 0)
# for j in range(len(x_data))[n_samples:]:
#     x_e = x_data[j].astype('float32')
#     preds = []
#     for adapter in adapters:
#         preds.append(adapter.convert(x_e))
#     pred_e = np.concatenate(preds, axis=1)
#     location_e = location_data[j].astype('float32')
#     resolution_e = resolution_data[j].astype('float32')
#     volume_e = volume_data[j].astype('float32')
#     loss_data = fusion_val_fn(pred_e, location_e, resolution_e, volume_e)
#     losses_data.append(loss_data)
# print("total #samples: {}, loss: {}".format(len(x_data), np.mean(losses_data)))

-----------------predicting---------------------
total #samples: 1000, loss: 0.0109496666119


# Save to submit

In [6]:
# bm.fusion_submit(adapters, fusion_area_fn, fusion_test_fn, fixed_size, 'submit.csv')